In [21]:
def get_asciiz(barray, pos):
    str=''
    while barray[pos] != 0:
        str += chr(barray[pos])
        pos += 1
    return str

def get_dword(barray, pos):
    val=0
    for i in range(3, -1, -1):
        val = val<<8 | barray[pos+i]
    return val

def get_word(barray, pos):
    val=0
    for i in range(1, -1, -1):
        val = val<<8 | barray[pos+i]
    return val

def get_header(img):
    image_name = get_asciiz(img, 0)
    write_protect = img[0x1a]
    disk_type = img[0x1b]
    disk_size = get_dword(img, 0x1c)
    hdr= { 'name'          : image_name, 
           'write_protect' : 'ON' if write_protect==0x10 else 'OFF',
           'disk_type'     : '2D' if disk_type==0 else '2DD' if disk_type==0x10 else '2HD',
           'disk_size'     : '0x'+format(disk_size, '08X')
         }
    return hdr

def get_sect(img, ofst):
    CHRN         = img[ofst:ofst+4]
    num_sec      = get_word(img, ofst + 0x04)
    density      = img[ofst + 0x06]
    address_mark = img[ofst + 0x07]
    status       = img[ofst + 0x08]
    pos          = get_word(img, ofst + 0x09)
    ext_sta      = img[ofst + 0x0b]
    size         = get_word(img, ofst + 0x0e)
    id_crc_val   = img[ofst + 0x0c]<<8 | img[ofst + 0x0d]
    data         = img[ofst:ofst+size+1]
    return { 'CHRN' : '{}:{}:{}:{}'.format(*CHRN),
              'num_sec' : num_sec,
              'density' : 'D' if density==0 else 'S',
              'status' : status,
              'am' : address_mark,
              'ext_sta': ext_sta,
              'id_crc': id_crc_val,
              'pos': pos,
              'size' : size,
              'data' : data }

def decode_d77(file):
    with open(file, 'rb') as f:
        img = f.read()

    d77 = {}

    # Extract Header information
    d77['header'] = get_header(img)

    # Extract track offset
    trk_ofst = [0]*164
    for t in range(164):
        trk_ofst[t] = get_dword(img, 0x20 + t*4)
        #print('{} {:08X}'.format(t, trk_ofst[t]))
    d77['trk_ofst'] = trk_ofst

    # Extract sector data
    disk = []
    for t in range(164):
        disk.append([])
        ofst = trk_ofst[t]
        if ofst == 0:
            continue
        sect = get_sect(img, ofst)
        num_sect = sect['num_sec']
        for i in range(num_sect):
            sect = get_sect(img, ofst)
            disk[-1].append(sect)
            ofst += 0x10 + sect['size']
    d77['disk'] = disk
    return d77

In [30]:
#d77 = decode_d77('putty/cdos7.d77')
d77 = decode_d77('copyboy7.d77')

# Display results
print('Header:', d77['header'])

disk = d77['disk']
for trkid, track in enumerate(disk):
    print(trkid)
    print('C  H  R  N    S1 S2   AM   SIZE ICRC')
    for sect in track:
        c,h,r,n = sect['CHRN'].split(':')
        status = sect['status']
        size = sect['size']
        am = 'DDAM' if sect['am']==0x10 else 'DAM '
        pos = sect['pos']
        ext_sta = sect['ext_sta']
        id_crc = sect['id_crc']  # id crc value
        print('{:02x} {:02x} {:02x} {:02x} - {:02x} {:02x} - {:4} {:4} {:04x}'.format(int(c),int(h),int(r),int(n),
                                                                                                  status, ext_sta, am, size, id_crc))
        #print('density', sect['density'])
        #print('num_sec', sect['num_sec'])

Header: {'name': 'DISK', 'write_protect': 'OFF', 'disk_type': '2D', 'disk_size': '0x000D9818'}
0
C  H  R  N    S1 S2   AM   SIZE ICRC
00 00 01 01 - 00 00 - DAM   256 fa0c
00 00 02 01 - 00 00 - DAM   256 af5f
00 00 03 01 - 00 00 - DAM   256 9c6e
00 00 04 01 - 00 00 - DAM   256 05f9
00 00 05 01 - 00 00 - DAM   256 36c8
00 00 06 01 - 00 00 - DAM   256 639b
00 00 07 01 - 00 00 - DAM   256 50aa
00 00 08 01 - 00 00 - DAM   256 4094
00 00 09 01 - 00 00 - DAM   256 73a5
00 00 0a 01 - 00 00 - DAM   256 26f6
00 00 0b 01 - 00 00 - DAM   256 15c7
00 00 0c 01 - 00 00 - DAM   256 8c50
00 00 0d 01 - 00 00 - DAM   256 bf61
00 00 0e 01 - 00 00 - DAM   256 ea32
00 00 0f 01 - 00 00 - DAM   256 d903
00 00 10 01 - 00 00 - DAM   256 ca4e
1
C  H  R  N    S1 S2   AM   SIZE ICRC
00 01 01 01 - 00 00 - DAM   256 cd3c
00 01 02 01 - 00 00 - DAM   256 986f
00 01 03 01 - 00 00 - DAM   256 ab5e
00 01 04 01 - 00 00 - DAM   256 32c9
00 01 05 01 - 00 00 - DAM   256 01f8
00 01 06 01 - 00 00 - DAM   256 54ab
00 01 07 01 -

In [ ]:
}